In [ ]:
%matplotlib inline
import os
import h5py
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils import data
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils import data

In [ ]:
from scripts.dataloaders import dataLoaderSeg

    
dataset = "BRATSmini5_train.h5"
# Getting the patient numbers
iDlist = []
with h5py.File(dataset, 'r' ) as f:
    keys = list(f.keys())
    for k in keys:
        # First split is patient iD
        iD = k.split("_")[0]
        if iD in iDlist:
            pass
        else:
            iDlist.append(iD)

Npat = len(iDlist)
trai_list = iDlist[0:Npat//3]
vali_list = iDlist[Npat//3:2*Npat//3]
test_list = iDlist[2*Npat//3:-1]


trai_data = dataLoaderSeg(dataset, trai_list, enableRotation=True)
vali_data = dataLoaderSeg(dataset, vali_list, enableRotation=True)
test_data = dataLoaderSeg(dataset, test_list)

In [ ]:
# U-Net parameters
wf = 16
depth = 4
in_chans= 1

# optimizastion
bs = 8
ilr = 3.5e-4
epo = 300

In [ ]:
from scripts.customLosses import wBCE, BinaryIoU
lossDice = BinaryIoU(smooth=5.0)

def loss_trai(cnn,x, y):
    
    segLoss = lossDice(cnn(x)[0], y)
    
    return(segLoss )

def loss_vali(cnn,x, y):
    return(lossDice(cnn(x)[0], y) )

In [ ]:
from scripts.training import trainingLoop
##############
# Epoch loop #
for run in np.arange(5):
    from CNNs.unet import unet_2d
    cnn = unet_2d(in_chans, depth, wf).float().cuda()
    trainingLoop(
        cnn,
        trai_data,
        vali_data,
        epo, bs,
        loss_trai,
        loss_vali,
        torch.optim.AdamW(cnn.parameters(), lr=ilr, weight_decay=0.25),
        ilr,
        "./CNN_weights/unet_run%s.pyt"%run,
        printLog=True,
        logName="./CNN_weights/logs/unet_run%s.log"%run)